In [1]:
import pyspark
import os

In [2]:
myConf= pyspark.SparkConf()

spark= pyspark.sql.SparkSession\
    .builder\
    .master('local')\
    .appName('myApp')\
    .config(conf=myConf)\
    .getOrCreate()

In [3]:
myList= [('김하나', "English", 100),
        ('김하나', "Math", 80),
        ('임하나', "English", 70),
        ('임하나', "Math", 100),
        ('김갑돌', "English", 82.3),
        ('김갑돌', "Math", 98.5)]

In [4]:
myRdd= spark.sparkContext\
    .parallelize(myList)

In [5]:
print('#partitions: ', myRdd.getNumPartitions())
myRdd.collect()

#partitions:  1


[('김하나', 'English', 100),
 ('김하나', 'Math', 80),
 ('임하나', 'English', 70),
 ('임하나', 'Math', 100),
 ('김갑돌', 'English', 82.3),
 ('김갑돌', 'Math', 98.5)]

### 문제 3-1

In [6]:
myRdd.map(lambda x: (x[0], float(x[2]))).collect()

[('김하나', 100.0),
 ('김하나', 80.0),
 ('임하나', 70.0),
 ('임하나', 100.0),
 ('김갑돌', 82.3),
 ('김갑돌', 98.5)]

In [7]:
name_sum= myRdd.map(lambda x: (x[0], float(x[2])))\
    .combineByKey(lambda value: (value,1),
                  lambda x, value: x[0]+value,
                  lambda x, y: x+y)\
    .sortByKey(False)

In [8]:
name_sum.collect()

[('임하나', 170.0), ('김하나', 180.0), ('김갑돌', 180.8)]

In [9]:
name_sum_print= name_sum.collect()
for i in name_sum_print:
    print("'"+i[0]+"'", i[1])

'임하나' 170.0
'김하나' 180.0
'김갑돌' 180.8


### 문제 3-2

In [10]:
# myList= [('김하나', "English", 100),
#         ('김하나', "Math", 80),
#         ('임하나', "English", 70),
#         ('임하나', "Math", 100),
#         ('김갑돌', "English", 82.3),
#         ('김갑돌', "Math", 98.5)]

In [11]:
subject_sum= myRdd.map(lambda x: (x[1], float(x[2])))\
    .combineByKey(lambda value: (value, 1),
                  lambda x, value: (x[0]+value, ),
                  lambda x, y: x+y)\
    .sortByKey(True)
subject_sum.collect()
#name_sum을 할 때는 mergeValue에 count를 채우지 않아도 돌아갔는데,
#subject_name을 할 때는 mergeValue에 count 자리를 마련해야만 돌아갑니다... 왜인지 모르겠습니다.

[('English', (252.3,)), ('Math', (278.5,))]

In [12]:
subject_sum_print= subject_sum.collect()
for i in subject_sum_print:
    print("'"+i[0]+"'", i[1][0])

'English' 252.3
'Math' 278.5


### 문제 3-3

In [13]:
name_sum_count= myRdd.map(lambda x: (x[0], float(x[2])))\
    .combineByKey(lambda value: (value,1),
                  lambda x, value: (x[0]+value, x[1]+1),
                  lambda x, y: x+y)\
    .sortByKey(False)

In [14]:
name_sum_count.collect()

[('임하나', (170.0, 2)), ('김하나', (180.0, 2)), ('김갑돌', (180.8, 2))]

In [15]:
name_sumCount_print= name_sum_count.collect()
for i in name_sumCount_print:
    print("'"+i[0]+"'", i[1])

'임하나' (170.0, 2)
'김하나' (180.0, 2)
'김갑돌' (180.8, 2)


### 문제 3-4

In [16]:
name_avg= name_sum_count.map(lambda x:(x[0], x[1][0]/x[1][1]))

In [17]:
name_avg.collect()

[('임하나', 85.0), ('김하나', 90.0), ('김갑돌', 90.4)]

In [18]:
name_avg_print= name_avg.collect()
for i in name_avg_print:
    print("'"+i[0]+"'", i[1])

'임하나' 85.0
'김하나' 90.0
'김갑돌' 90.4
